In [4]:
import pandas as pd
import numpy as np
import requests
import os 
from dotenv import load_dotenv
import requests
from tqdm import tqdm
import pickle
pd.options.display.max_columns = None

# Importing pickle file

In [6]:
with open('../data/pickle/spotify_dataset.pickle', 'rb') as tracks:
    tracks = pickle.load(tracks)

In [7]:
tracks.head()

,artist_clean,track_clean
0,elvis costello,(the angels wanna wear my) red shoes
1,elvis costello and the attractions,"(what's so funny 'bout) peace, love and unders..."
2,tiffany page,7 years too late
3,elvis costello and the attractions,accidents will happen
4,elvis costello,alison


# lastfm
In this part of the process I will extract form the LastFM API all the data related to the tracks and artist genres.

First, I need to configure my credentials.

In [8]:
load_dotenv()

True

In [9]:
api = os.getenv("lastfm-id")
username = os.getenv("lastfm-user")

In [10]:
## me voy a hacer una función con los datos de la petición
def getLastFMData(params):
    headers = {'user-agent': username} ## defino mi usuario
    url = 'https://ws.audioscrobbler.com/2.0/' ##url a la que voy a contactar

    ## configuro los params para hacer la petición
    params['api_key'] = api
    params['format'] = 'json'

    res = requests.get(url, headers=headers, params=params)
    return res

In [49]:
res = getLastFMData({
    'method': 'artist.getInfo',
    'artist':'Harry Styles',
})
res.json()['artist']['bio']#['sumary']#[0]['name']

{'links': {'link': {'#text': '',
   'rel': 'original',
   'href': 'https://last.fm/music/Harry+Styles/+wiki'}},
 'published': '24 Aug 2011, 17:11',
 'summary': 'Harry Edward Styles (born 1 February 1994 in Redditch, Worcestershire, England) is a Grammy-winning, British singer, songwriter, and actor. As a member of the British (&Irish) boy band One Direction, singer Harry Styles topped the charts, toured the world, and sold millions of albums before going solo in 2016. Styles was raised in Holmes Chapel, Cheshire, and he made his first foray into music with a high-school band named White Eskimo. In 2010, he left home and joined the seventh season of The X Factor <a href="https://www.last.fm/music/Harry+Styles">Read more on Last.fm</a>',
 'content': 'Harry Edward Styles (born 1 February 1994 in Redditch, Worcestershire, England) is a Grammy-winning, British singer, songwriter, and actor. As a member of the British (&Irish) boy band One Direction, singer Harry Styles topped the charts, to

In [45]:
def getArtistInfo(artist):
    ## hago la colsulta para obtener los datos de tags por artista
    res = getLastFMData({
        'method': 'artist.getInfo',
        'artist':  artist,
    })

    # por si me da un timeout
    if res.status_code != 200:
        return None

    # extraigo los top 5 tags de cada artistas
    try:
        #tags = [tag['name'] for tag in res.json()['toptags']['tag'][:3]]
        return res.json()['artist']['bio'], res.json()['artist']['tags']
    except:
        return np.nan

First I try on a slice of my dataframe

In [50]:
tqdm.pandas()

test = tracks.sample(100)
test[['bio', 'artist_tag']] = test.progress_apply(lambda x: getArtistInfo(x.artist_clean), axis=1, result_type='expand')

100%|██████████| 100/100 [00:19<00:00,  5.01it/s]


In [47]:
test.head()

,artist_clean,track_clean,bio,tags
1298294,meye mind,bitch,"{'links': {'link': {'#text': '', 'rel': 'origi...",{'tag': []}
795121,the kossoy sisters,bowling green,"{'links': {'link': {'#text': '', 'rel': 'origi...","{'tag': [{'name': 'folk', 'url': 'https://www...."
9541961,days of the new,skeleton key,"{'links': {'link': {'#text': '', 'rel': 'origi...","{'tag': [{'name': 'Grunge', 'url': 'https://ww..."
65729,collegium aureum,"pavane mille regretz""/ronde/pavane ""si pas sou...","{'links': {'link': {'#text': '', 'rel': 'origi...","{'tag': [{'name': 'Classical', 'url': 'https:/..."
12312328,arr. j mendelsohn,piano concerto no.21 in c (andante),"{'links': {'link': {'#text': '', 'rel': 'origi...",{'tag': []}


This is for the whole dataframe:

In [62]:
tqdm.pandas()

#test = tracks.sample(100)
tracks[['bio', 'artist_tag']] = tracks.progress_apply(lambda x: getArtistInfo(x.artist_clean), axis=1, result_type='expand')

  0%|          | 35/2691039 [00:06<128:18:04,  5.83it/s]


KeyboardInterrupt: 

In [ ]:
tracks.head()

,artist_clean,track_clean,bio,tags
1298294,meye mind,bitch,"{'links': {'link': {'#text': '', 'rel': 'origi...",{'tag': []}
795121,the kossoy sisters,bowling green,"{'links': {'link': {'#text': '', 'rel': 'origi...","{'tag': [{'name': 'folk', 'url': 'https://www...."
9541961,days of the new,skeleton key,"{'links': {'link': {'#text': '', 'rel': 'origi...","{'tag': [{'name': 'Grunge', 'url': 'https://ww..."
65729,collegium aureum,"pavane mille regretz""/ronde/pavane ""si pas sou...","{'links': {'link': {'#text': '', 'rel': 'origi...","{'tag': [{'name': 'Classical', 'url': 'https:/..."
12312328,arr. j mendelsohn,piano concerto no.21 in c (andante),"{'links': {'link': {'#text': '', 'rel': 'origi...",{'tag': []}


Now I get the tags for each individual track:

In [55]:
def getTrackTags(artist, track):
    ## hago la colsulta para obtener los datos de tags por artista
    res = getLastFMData({
        'method': 'track.getTopTags',
        'artist':  artist,
        'track': track,
    })

    # por si me da un timeout
    if res.status_code != 200:
        return None

    # extraigo los top 5 tags de cada artistas
    try:
        #tags = [tag['name'] for tag in res.json()['toptags']['tag'][:3]]
        return [tag['name'] for tag in res.json()['toptags']['tag'][:3]] ## no va muy bien esta parte y solo me extrae tres géneros... 
    except:
        return np.nan

This is how I will get info for a slice of my dataframe:

In [56]:
tqdm.pandas()

test = tracks.sample(100)
test['track_tags'] = test.progress_apply(lambda x: getTrackTags(x.artist_clean, x.track_clean), axis=1)

100%|██████████| 100/100 [00:17<00:00,  5.87it/s]


In [60]:
print(test['track_tags'].isnull().sum())
test.head()

24


,artist_clean,track_clean,track_tags
2300027,neil young,stayin' power/remastered,NaN
12562223,carla magnan,voci,[]
7811577,saint etienne,who do you think you are?/roger sanchez mix,NaN
7617983,ringo starr,loser's lounge,"[pop, catchy, oldies]"
4901307,maggie bell,as the years go passing by,"[female vocalists, blues rock, malone]"


This is for the whole dataset:

In [61]:
tqdm.pandas()

#test = tracks.sample(100)
tracks['track_tags'] = tracks.progress_apply(lambda x: getTrackTags(x.artist_clean, x.track_clean), axis=1)

  0%|          | 218/2691039 [00:37<128:53:27,  5.80it/s]


KeyboardInterrupt: 

In [ ]:
print(tracks['track_tags'].isnull().sum())
tracks.head()

24


,artist_clean,track_clean,track_tags
2300027,neil young,stayin' power/remastered,NaN
12562223,carla magnan,voci,[]
7811577,saint etienne,who do you think you are?/roger sanchez mix,NaN
7617983,ringo starr,loser's lounge,"[pop, catchy, oldies]"
4901307,maggie bell,as the years go passing by,"[female vocalists, blues rock, malone]"


# Export

In [ ]:
with open('../data/spotify_dataset_lastfm.pickle', 'wb') as data_lastfm:
    pickle.dump(tracks, data_lastfm)